In [1]:
from collections import defaultdict
from dataclasses import dataclass
from functools import reduce

import aocd
import parse

In [2]:
@dataclass
class Sensor:
    coords: tuple[int, int]
    closest_beacon: tuple[int, int]

    @property
    def beacon_distance(self) -> int:
        return abs(self.coords[0] - self.closest_beacon[0]) + abs(self.coords[1] - self.closest_beacon[1])

lines = aocd.get_data(day=15, year=2022).splitlines()
sensors = [Sensor(*(tup.fixed for tup in parse.findall('x={:d}, y={:d}', line))) for line in lines]

In [3]:
Ranges = dict[int, list[range]]
beacons:    Ranges = defaultdict(list)
no_beacons: Ranges = defaultdict(list)

for sensor in sensors:
    x, y = sensor.coords
    dist = sensor.beacon_distance

    for offset in range(-dist, dist+1):
        beacon_x, beacon_y = sensor.closest_beacon
        if beacon_y == y+offset:
            beacons[beacon_y].append(range(beacon_x, beacon_x+1))
            if beacon_x < x:
                start = x-dist+abs(offset)+1
                stop  = x+dist-abs(offset)+1
            else:
                start = x-dist+abs(offset)
                stop  = x+dist-abs(offset)
        else:
            start     = x-dist+abs(offset)
            stop      = x+dist-abs(offset)+1
        no_beacons[y+offset].append(range(start, stop))

In [4]:
print("Part 1:", len(reduce(set.union, [set(r) for r in no_beacons[2000000]], set())))

Part 1: 4725496


In [5]:
for y in range(4000001):
    possible = range(4000001)
    not_possible = no_beacons[y] + beacons[y]
    while len(possible) > 1:
        for np in not_possible:
            if np.start <= possible.start and np.stop > possible.start:
                possible = range(np.stop, possible.stop)
            if np.stop >= possible.stop and np.start < possible.stop:
                possible = range(possible.start, np.start)
    if len(possible) == 1:
        for np in not_possible:
            if np.start <= possible.start and np.stop > possible.start:
                possible = range(np.stop, possible.stop)
            if np.stop >= possible.stop and np.start < possible.stop:
                possible = range(possible.start, np.start)
    if len(possible) == 1:
        print("Part 2:", possible.start*4000000+y)
        break

Part 2: 12051287042458
